In [ ]:
"""This is a sample code submitted for hackathon verification as requested by organizers. To prevent potential leaks, sensitive keys and methods are not shared
as i was told by the lead organiser that project leak can not be guaranteed."""


{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 42,
   "id": "0e8c090a",
   "metadata": {},
   "outputs": [],
   "source": [
    "import pyaudio\n",
    "import librosa\n",
    "import numpy as np\n",
    "import tensorflow as tf\n",
    "from queue import Queue\n",
    "import time\n",
    "from threading import Thread"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 43,
   "id": "2ad0169a",
   "metadata": {},
   "outputs": [],
   "source": [
    "AUDIO_RATE = 44100\n",
    "AUDIO_DURATION = 2  # seconds\n",
    "AUDIO_FORMAT = pyaudio.paFloat32\n",
    "AUDIO_CHANNELS = 1\n",
    "FRAMES_PER_BUFFER = int(AUDIO_RATE * 0.1)\n",
    "THRESHOLD = 0.01\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 44,
   "id": "a0881594",
   "metadata": {},
   "outputs": [],
   "source": [
    "MODEL_PATH = \"gunshot_model_1D.tflite\"\n",
    "interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)\n",
    "interpreter.allocate_tensors()\n",
    "input_details = interpreter.get_input_details()\n",
    "output_details = interpreter.get_output_details()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 45,
   "id": "61467085",
   "metadata": {},
   "outputs": [],
   "source": [
    "audio_queue = Queue()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 46,
   "id": "77ece67f",
   "metadata": {},
   "outputs": [],
   "source": [
    "def callback(in_data, frame_count, time_info, status):\n",
    "    samples = np.frombuffer(in_data, dtype=np.float32)\n",
    "    callback.buffer = np.append(callback.buffer, samples)\n",
    "    if len(callback.buffer) >= AUDIO_RATE * AUDIO_DURATION:\n",
    "        audio_queue.put(callback.buffer[:AUDIO_RATE * AUDIO_DURATION])\n",
    "        callback.buffer = callback.buffer[AUDIO_RATE * AUDIO_DURATION:]\n",
    "    return (in_data, pyaudio.paContinue)\n",
    "callback.buffer = np.zeros(0, dtype=np.float32)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 47,
   "id": "d1e1efa9",
   "metadata": {},
   "outputs": [],
   "source": [
    "def audio_processing_thread():\n",
    "    while True:\n",
    "        audio = audio_queue.get()\n",
    "        if np.max(np.abs(audio)) < THRESHOLD:\n",
    "            continue\n",
    "\n",
    "        # Resample to match training, if needed\n",
    "        audio = librosa.resample(audio, orig_sr=AUDIO_RATE, target_sr=22050)\n",
    "        audio = np.pad(audio, (0, max(0, 44100 - len(audio))))[:44100].reshape(44100, 1).astype(np.float32)\n",
    "\n",
    "        # Inference\n",
    "        interpreter.set_tensor(input_details[0]['index'], audio[np.newaxis, ...])\n",
    "        interpreter.invoke()\n",
    "        output = interpreter.get_tensor(output_details[0]['index'])[0]\n",
    "\n",
    "        gunshot_confidence = output[0]  # Assuming output[0] = gunshot\n",
    "        if gunshot_confidence >= 0.8:\n",
    "            print(\"🔥 Gunshot Detected! Confidence:\", gunshot_confidence)\n",
    "        else:\n",
    "            print(\"Other / Uncertain. Confidence:\", gunshot_confidence)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 48,
   "id": "a02a8462",
   "metadata": {},
   "outputs": [],
   "source": [
    "pa = pyaudio.PyAudio()\n",
    "stream = pa.open(format=AUDIO_FORMAT,\n",
    "                 channels=AUDIO_CHANNELS,\n",
    "                 rate=AUDIO_RATE,\n",
    "                 input=True,\n",
    "                 frames_per_buffer=FRAMES_PER_BUFFER,\n",
    "                 stream_callback=callback)\n",
    "stream.start_stream()\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "16fe448b",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Other / Uncertain. Confidence: 0.47605637\n",
      "Other / Uncertain. Confidence: 0.7990551\n",
      "Other / Uncertain. Confidence: 0.6881857\n",
      "Other / Uncertain. Confidence: 0.73991114\n",
      "🔥 Gunshot Detected! Confidence: 0.8305648\n",
      "Other / Uncertain. Confidence: 0.48505476\n"
     ]
    }
   ],
   "source": [
    "thread = Thread(target=audio_processing_thread)\n",
    "thread.daemon = True\n",
    "thread.start()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 50,
   "id": "a3d5362c",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Listening for gunshots (1D model only)...\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Stopped.\n"
     ]
    }
   ],
   "source": [
    "print(\"Listening for gunshots (1D model only)...\")\n",
    "try:\n",
    "    while True:\n",
    "        time.sleep(1)\n",
    "except KeyboardInterrupt:\n",
    "    stream.stop_stream()\n",
    "    stream.close()\n",
    "    pa.terminate()\n",
    "    print(\"Stopped.\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "base",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.16"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}